In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install nibabel

In [ ]:
pip install split-folders

In [ ]:
import nibabel as nib

In [ ]:
# TRAIN_DATASET_PATH = 'BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'
#VALIDATION_DATASET_PATH = 'BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'
# TRAIN_DATASET_PATH = "../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/"
# path_masks = "../input/brain-tumor-dataset-with-saliency/masks"

# test_image_flair=nib.load(TRAIN_DATASET_PATH + 'BraTS20_Training_100/BraTS20_Training_100_flair.nii').get_fdata()
# print(test_image_flair.max())

In [ ]:
pip install MedPy

In [ ]:
pip install --upgrade scipy --user

In [ ]:
"""
All BraTS multimodal scans are available as NIfTI files (.nii.gz) -> commonly used medical imaging format to store brain imagin data obtained using MRI and describe different MRI settings
T1: T1-weighted, native image, sagittal or axial 2D acquisitions, with 1–6 mm slice thickness.
T1ce: T1-weighted, contrast-enhanced (Gadolinium) image, with 3D acquisition and 1 mm isotropic voxel size for most patients.
T2: T2-weighted image, axial 2D acquisition, with 2–6 mm slice thickness.
FLAIR: T2-weighted FLAIR image, axial, coronal, or sagittal 2D acquisitions, 2–6 mm slice thickness.
#Note: Segmented file name in Folder 355 has a weird name. Rename it to match others.
"""
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import nibabel as nib
import glob
import random
# from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tifffile import imsave
from skimage import img_as_float
from skimage.metrics import peak_signal_noise_ratio
from matplotlib import pyplot as plt
from skimage import io
from scipy import ndimage as nd
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,denoise_wavelet, estimate_sigma)
from medpy.filter.smoothing import anisotropic_diffusion
from scipy.signal import wiener
from sklearn.preprocessing import MinMaxScaler
from skimage.metrics import mean_squared_error
from skimage.metrics import structural_similarity as ssim
from scipy.signal import convolve2d as conv2
from skimage import color, data, restoration
from PIL import Image
scaler = MinMaxScaler()








##########################
#This part of the code to get an initial understanding of the dataset.
#################################
#PART 1: Load sample images and visualize
#Includes, dividing each image by its max to scale them to [0,1]
#Converting mask from float to uint8
#Changing mask pixel values (labels) from 4 to 3 (as the original labels are 0, 1, 2, 4)
#Visualize
###########################################
#View a few images

#Note: Segmented file name in Folder 355 has a weird name. Rename it to match others.


# Setting up of image path and scaling the image

# TRAIN_DATASET_PATH = 'BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'
TRAIN_DATASET_PATH = "../input/brats-2d/Brats_2d/images_t1ce/"
#VALIDATION_DATASET_PATH = 'BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'


gaussian_cleaned_psnr_t1ce=[]
gaussian_cleaned_mse_t1ce=[]
gaussian_cleaned_ssim_t1ce=[]

bilateral_cleaned_psnr_t1ce=[]
bilateral_cleaned_mse_t1ce=[]
bilateral_cleaned_ssim_t1ce=[]

aniso_cleaned_psnr_t1ce=[]
aniso_cleaned_mse_t1ce=[]
aniso_cleaned_ssim_t1ce=[]

wiener_cleaned_psnr_t1ce=[]
wiener_cleaned_mse_t1ce=[]
wiener_cleaned_ssim_t1ce=[]

gaussian_img_t1ce=[]
bilateral_img_t1ce=[]
img_aniso_filtered_t1ce=[]

k=0


# Put the values from 10000 for m
m=10000
n=30500
deconvolved_t1ce=[None] * (n-m)
    
for i in range(m,n):
    test_image_t1ce=np.asarray(Image.open(TRAIN_DATASET_PATH + "{}_t1ce.png".format(i)))
    test_image_t1ce=scaler.fit_transform(test_image_t1ce.reshape(-1, test_image_t1ce.shape[-1])).reshape(test_image_t1ce.shape)

    
    # img_as_float converts an image from any type to a floating point type in [0, 1], whether it is a color image or not. For example, a (512, 512) image containing uint8 values in the range 0-255 will be converted to a (512, 512) image containing float64 values in the range 0-1.
    #converting the image to float type. Since mathematical calculations are required on the image and also mostly, the skimage functions accepts float type image.
    noisy_image_t1ce=img_as_float(test_image_t1ce)
#     print(test_image_t1ce.shape)




    # DENOISING FILTERS
    gaussian_img_t1ce.append(nd.gaussian_filter(noisy_image_t1ce, sigma=5))
    bilateral_img_t1ce.append(denoise_bilateral(noisy_image_t1ce,sigma_spatial=15))
    img_aniso_filtered_t1ce.append(anisotropic_diffusion(noisy_image_t1ce, niter=50, kappa=20, gamma=0.2, option=1))


    rng = np.random.default_rng()
    psf = np.ones((5, 5)) / 25

    t1ce = test_image_t1ce
    t1ce = conv2(t1ce, psf, 'same')
    t1ce += 0.1 * t1ce.std() * rng.standard_normal(t1ce.shape)

    deconvolved_t1ce[k],_=restoration.unsupervised_wiener(t1ce, psf)






    # Gaussian
    
    gaussian_cleaned_psnr_t1ce.append(peak_signal_noise_ratio(test_image_t1ce, np.array(gaussian_img_t1ce[k])))
    gaussian_cleaned_mse_t1ce.append(mean_squared_error(test_image_t1ce, np.array(gaussian_img_t1ce[k])))
    gaussian_cleaned_ssim_t1ce.append(ssim(test_image_t1ce, np.array(gaussian_img_t1ce[k]), multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
#      gaussian_cleaned_psnr_t1ce.append(peak_signal_noise_ratio(test_image_t1ce[:,:,n_slice], gaussian_img_t1ce[:,:,n_slice]))
#     gaussian_cleaned_mse_t1ce.append(mean_squared_error(test_image_t1ce[:,:,n_slice], gaussian_img_t1ce[:,:,n_slice]))
#     gaussian_cleaned_ssim_t1ce.append(ssim(test_image_t1ce[:,:,n_slice], gaussian_img_t1ce[:,:,n_slice], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    
    #Bilateral
    bilateral_cleaned_psnr_t1ce.append(peak_signal_noise_ratio(test_image_t1ce,np.array(bilateral_img_t1ce[k])))
    bilateral_cleaned_mse_t1ce.append(mean_squared_error(test_image_t1ce, np.array(bilateral_img_t1ce[k])))
    bilateral_cleaned_ssim_t1ce.append(ssim(test_image_t1ce, np.array(bilateral_img_t1ce[k]), multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    
    #Anisotropic
    aniso_cleaned_psnr_t1ce.append(peak_signal_noise_ratio(test_image_t1ce, np.array(img_aniso_filtered_t1ce[k])))
    aniso_cleaned_mse_t1ce.append(mean_squared_error(test_image_t1ce, np.array(img_aniso_filtered_t1ce[k])))
    aniso_cleaned_ssim_t1ce.append(ssim(test_image_t1ce, np.array(img_aniso_filtered_t1ce[k]), multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    #Wiener
    wiener_cleaned_psnr_t1ce.append(peak_signal_noise_ratio(test_image_t1ce,np.array(deconvolved_t1ce[k])))
    wiener_cleaned_mse_t1ce.append(mean_squared_error(test_image_t1ce, np.array(deconvolved_t1ce[k])))
    wiener_cleaned_ssim_t1ce.append(ssim(test_image_t1ce, np.array(deconvolved_t1ce[k]), multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))



    k+=1

gaussian_sum_psnr_t1ce=0
gaussian_sum_mse_t1ce=0
gaussian_sum_ssim_t1ce=0


bilateral_sum_psnr_t1ce=0
bilateral_sum_mse_t1ce=0
bilateral_sum_ssim_t1ce=0


aniso_sum_psnr_t1ce=0
aniso_sum_mse_t1ce=0
aniso_sum_ssim_t1ce=0


wiener_sum_psnr_t1ce=0
wiener_sum_mse_t1ce=0
wiener_sum_ssim_t1ce=0

for i in range(0,(n-m)):
    gaussian_sum_psnr_t1ce +=gaussian_cleaned_psnr_t1ce[i]
    gaussian_sum_mse_t1ce +=gaussian_cleaned_mse_t1ce[i]
    gaussian_sum_ssim_t1ce +=gaussian_cleaned_ssim_t1ce[i]
    
    bilateral_sum_psnr_t1ce +=bilateral_cleaned_psnr_t1ce[i]
    bilateral_sum_mse_t1ce +=bilateral_cleaned_mse_t1ce[i]
    bilateral_sum_ssim_t1ce +=bilateral_cleaned_ssim_t1ce[i]
    
    aniso_sum_psnr_t1ce +=aniso_cleaned_psnr_t1ce[i]
    aniso_sum_mse_t1ce +=aniso_cleaned_mse_t1ce[i]
    aniso_sum_ssim_t1ce +=aniso_cleaned_ssim_t1ce[i]
    
    wiener_sum_psnr_t1ce +=wiener_cleaned_psnr_t1ce[i]
    wiener_sum_mse_t1ce +=wiener_cleaned_mse_t1ce[i]
    wiener_sum_ssim_t1ce +=wiener_cleaned_ssim_t1ce[i]

# print("\n"*2)


# print("-"*10 + "T1CE" + "-"*10 + "\n"*2)
# print(" "*10 + "GAUSSIAN" + " "*10 + "\n")
# print(" "*5 + "psnr: {}".format(gaussian_sum_psnr_t1ce/(n-m)))
# print(" "*5 + "mse: {}".format(gaussian_sum_mse_t1ce/(n-m)))
# print(" "*5 + "ssim: {}".format(gaussian_sum_ssim_t1ce/(n-m)))

# print("\n"*2 + " "*10 + "BILATERAL" + " "*10 + "\n")
# print(" "*5 + "psnr: {}".format(bilateral_sum_psnr_t1ce/(n-m)))
# print(" "*5 + "mse: {}".format(bilateral_sum_mse_t1ce/(n-m)))
# print(" "*5 + "ssim: {}".format(bilateral_sum_ssim_t1ce/(n-m)))

# print("\n"*2 + " "*10 + "ANISOTROPIC" + " "*10 + "\n")
# print(" "*5 + "psnr: {}".format(aniso_sum_psnr_t1ce/(n-m)))
# print(" "*5 + "mse: {}".format(aniso_sum_mse_t1ce/(n-m)))
# print(" "*5 + "ssim: {}".format(aniso_sum_ssim_t1ce/(n-m)))

# print("\n"*2 + " "*10 + "WIENER" + " "*10 + "\n")
# print(" "*5 + "psnr: {}".format(wiener_sum_psnr_t1ce/(n-m)))
# print(" "*5 + "mse: {}".format(wiener_sum_mse_t1ce/(n-m)))
# print(" "*5 + "ssim: {}".format(wiener_sum_ssim_t1ce/(n-m)))

print("\n"*2)

In [ ]:
"""
All BraTS multimodal scans are available as NIfTI files (.nii.gz) -> commonly used medical imaging format to store brain imagin data obtained using MRI and describe different MRI settings
T1: T1-weighted, native image, sagittal or axial 2D acquisitions, with 1–6 mm slice thickness.
T1ce: T1-weighted, contrast-enhanced (Gadolinium) image, with 3D acquisition and 1 mm isotropic voxel size for most patients.
T2: T2-weighted image, axial 2D acquisition, with 2–6 mm slice thickness.
FLAIR: T2-weighted FLAIR image, axial, coronal, or sagittal 2D acquisitions, 2–6 mm slice thickness.
#Note: Segmented file name in Folder 355 has a weird name. Rename it to match others.
"""
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import nibabel as nib
import glob
import random
# from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tifffile import imsave
from skimage import img_as_float
from skimage.metrics import peak_signal_noise_ratio
from matplotlib import pyplot as plt
from skimage import io
from scipy import ndimage as nd
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,denoise_wavelet, estimate_sigma)
from medpy.filter.smoothing import anisotropic_diffusion
from scipy.signal import wiener
from sklearn.preprocessing import MinMaxScaler
from skimage.metrics import mean_squared_error
from skimage.metrics import structural_similarity as ssim
from PIL import Image
scaler = MinMaxScaler()








##########################
#This part of the code to get an initial understanding of the dataset.
#################################
#PART 1: Load sample images and visualize
#Includes, dividing each image by its max to scale them to [0,1]
#Converting mask from float to uint8
#Changing mask pixel values (labels) from 4 to 3 (as the original labels are 0, 1, 2, 4)
#Visualize
###########################################
#View a few images

#Note: Segmented file name in Folder 355 has a weird name. Rename it to match others.


# Setting up of image path and scaling the image
gaussian_img_t1ce=[]
bilateral_img_t1ce=[]
img_aniso_filtered_t1ce=[]

k=0


# Put the values from 10000 for m
m=10000
n=10002
deconvolved_t1ce=[None] * (n-m)
    
for i in range(m,n):
    test_image_t1ce=np.asarray(Image.open(TRAIN_DATASET_PATH + "{}_t1ce.png".format(i)))
    test_image_t1ce=scaler.fit_transform(test_image_t1ce.reshape(-1, test_image_t1ce.shape[-1])).reshape(test_image_t1ce.shape)

    
    # img_as_float converts an image from any type to a floating point type in [0, 1], whether it is a color image or not. For example, a (512, 512) image containing uint8 values in the range 0-255 will be converted to a (512, 512) image containing float64 values in the range 0-1.
    #converting the image to float type. Since mathematical calculations are required on the image and also mostly, the skimage functions accepts float type image.
    noisy_image_t1ce=img_as_float(test_image_t1ce)
    print(test_image_t1ce.shape)




    # DENOISING FILTERS
    gaussian_img_t1ce.append(nd.gaussian_filter(noisy_image_t1ce, sigma=5))
    bilateral_img_t1ce.append(denoise_bilateral(noisy_image_t1ce,sigma_spatial=15))
    img_aniso_filtered_t1ce.append(anisotropic_diffusion(noisy_image_t1ce, niter=50, kappa=20, gamma=0.2, option=1))


    rng = np.random.default_rng()
    psf = np.ones((5, 5)) / 25

    t1ce = test_image_t1ce
    t1ce = conv2(t1ce, psf, 'same')
    t1ce += 0.1 * t1ce.std() * rng.standard_normal(t1ce.shape)

    deconvolved_t1ce[k],_=restoration.unsupervised_wiener(t1ce, psf)









    # Gaussian
    print("\n"*2)
    plt.figure(figsize=(12, 8))
    print("Gaussian")
    plt.subplot(233)
    plt.imshow(gaussian_img_t1ce[k], cmap='gray')
    gaussian_cleaned_psnr = peak_signal_noise_ratio(test_image_t1ce, gaussian_img_t1ce[k])
    gaussian_cleaned_mse = mean_squared_error(test_image_t1ce, gaussian_img_t1ce[k])
    gaussian_cleaned_ssim= ssim(test_image_t1ce, gaussian_img_t1ce[k], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
    print("PSNR of cleaned image = ", gaussian_cleaned_psnr)
    print("MSE of cleaned image = ", gaussian_cleaned_mse)
    print("SSIM of cleaned image = ", gaussian_cleaned_ssim)
    print("\n"*2)

    plt.show()



    # Bilateral

    plt.figure(figsize=(12, 8))
    print("Bilateral")
    plt.subplot(233)
    plt.imshow(bilateral_img_t1ce[k], cmap='gray')
    bilateral_cleaned_psnr = peak_signal_noise_ratio(test_image_t1ce,bilateral_img_t1ce[k])
    bilateral_cleaned_mse = mean_squared_error(test_image_t1ce, bilateral_img_t1ce[k])
    bilateral_cleaned_ssim= ssim(test_image_t1ce, bilateral_img_t1ce[k], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
    print("PSNR of cleaned image = ", bilateral_cleaned_psnr)
    print("MSE of cleaned image = ", bilateral_cleaned_mse)
    print("SSIM of cleaned image = ", bilateral_cleaned_ssim)
    print("\n"*2)

    plt.show()



    #Anistropic

    plt.figure(figsize=(12, 8))
    print("Anistropic")
    plt.subplot(233)
    plt.imshow(img_aniso_filtered_t1ce[k], cmap='gray')
    aniso_cleaned_psnr = peak_signal_noise_ratio(test_image_t1ce, img_aniso_filtered_t1ce[k])
    aniso_cleaned_mse = mean_squared_error(test_image_t1ce, img_aniso_filtered_t1ce[k])
    aniso_cleaned_ssim= ssim(test_image_t1ce, img_aniso_filtered_t1ce[k], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
    print("PSNR of cleaned image = ", aniso_cleaned_psnr)
    print("MSE of cleaned image = ", aniso_cleaned_mse)
    print("SSIM of cleaned image = ", aniso_cleaned_ssim)
    print("\n"*2)

    plt.show()



    # Wiener
    plt.figure(figsize=(12, 8))
    print("Wiener")
    plt.subplot(233)
    plt.imshow(deconvolved_t1ce[k], cmap='gray')
    wiener_cleaned_psnr = peak_signal_noise_ratio(test_image_t1ce,deconvolved_t1ce[k])
    wiener_cleaned_mse = mean_squared_error(test_image_t1ce, deconvolved_t1ce[k])
    wiener_cleaned_ssim= ssim(test_image_t1ce, deconvolved_t1ce[k], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
    print("PSNR of cleaned image = ", wiener_cleaned_psnr)
    print("MSE of cleaned image = ", wiener_cleaned_mse)
    print("SSIM of cleaned image = ", wiener_cleaned_ssim)
    print("\n"*2)

    plt.show()



    # Raw Images

    print("\n"*4)
    print("RAW IMAGES")
    plt.figure(figsize=(12, 8))
    plt.subplot(233)
    plt.imshow(test_image_t1ce, cmap='gray')
    plt.title('Image t1ce')
    plt.show()
    
    k+=1